In [1]:
# Revise conditional probabilty from here - 
#https://www.youtube.com/watch?v=9wCnvr7Xw4E


In [3]:
# P(A|B) = (P(B|A) * P(A))/P(B)

# - P(B|A) - Likelihood
# - P(A) - Prior prob of A 
# - P(B) - prob of given B knowledge - Normalisation - Marginal probability
# - P(A|B) - posterior probability

In [4]:
# Assumption Naive Bayes Classifier makes on top of Bayes Theorem
# All features are independent to each other

- Naive Bayes
- Mushroom Dataset
- Goal is to predict the class of mushrooms, given some features of the mushrooms. We will use Naive Bayes Model for this classification

1. Load the dataset

In [23]:
import numpy as np 
import pandas as pd

df = pd.read_csv('./mushrooms.csv')

In [24]:
print(df.head())
print(df.shape)

  type cap_shape cap_surface cap_color bruises odor gill_attachment  \
0    p         x           s         n       t    p               f   
1    e         x           s         y       t    a               f   
2    e         b           s         w       t    l               f   
3    p         x           y         w       t    p               f   
4    e         x           s         g       f    n               f   

  gill_spacing gill_size gill_color  ... stalk_surface_below_ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk_color_above_ring stalk_color_below_ring veil_type veil_color  \
0                      w                      w         p          w   
1             

In [43]:
data = df.values
print(np.unique(data[:,0]))
print(np.unique(data[:,1]))
print(np.unique(data[:,2]))
print(np.unique(data[:,3]))
print(np.unique(data[:,4]))
print(np.unique(data[:,21]))
print(np.unique(data[:,22]))

['e' 'p']
['b' 'c' 'f' 'k' 's' 'x']
['f' 'g' 's' 'y']
['b' 'c' 'e' 'g' 'n' 'p' 'r' 'u' 'w' 'y']
['f' 't']
['a' 'c' 'n' 's' 'v' 'y']
['d' 'g' 'l' 'm' 'p' 'u' 'w']


2. Encode the categorial data into numerical data

- We can do this by:
- FOR each column in dataset, 1. create a dictionary 2. then FOR each entry in the given column, 2.1 assign unique numerical values to the given alphabet.
- OR,
- USE skikit-learn

In [56]:
from sklearn.preprocessing import LabelEncoder

ls = LabelEncoder() # ls.fit_transform() returns encoded labels
# Below - apply transformation to each column - apply a operation on pandas dataframe that operation is specified in a function - which in this case is ls.fit_transform()
df_encoded = df.apply(ls.fit_transform)
print(type(df_encoded))
feature22 = df_encoded.iloc[:,22]
print(np.unique(feature22))
feature22_inverse = ls.inverse_transform(df_encoded.iloc[:,22]) # Note: LabelEncoder.fit applied to a dataframe, LabelEncoder class holds the last fitted column labels only.
print(feature22_inverse)
print(np.unique(feature22_inverse))

<class 'pandas.core.frame.DataFrame'>
[0 1 2 3 4 5 6]
['u' 'g' 'm' ... 'l' 'l' 'l']
['d' 'g' 'l' 'm' 'p' 'u' 'w']


In [57]:

data_encoded = df_encoded.values
print(np.unique(data_encoded[:,0]))
print(np.unique(data_encoded[:,1]))
print(np.unique(data_encoded[:,2]))
print(np.unique(data_encoded[:,3]))
print(np.unique(data_encoded[:,21]))
print(np.unique(data_encoded[:,22]))

[0 1]
[0 1 2 3 4 5]
[0 1 2 3]
[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5]
[0 1 2 3 4 5 6]


In [61]:
print(data_encoded[:5,:])
print(data_encoded.shape)

[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]
(8124, 23)


3. Break the data into train and test

In [65]:
from sklearn.model_selection import train_test_split

x = data_encoded[:,1:]
y = data_encoded[:,0]
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(6499, 22)
(6499,)
(1625, 22)
(1625,)


In [67]:
# Number of types of mushrooms
np.unique(ytrain)

array([0, 1])

4. Building our classifier

In [76]:
a = np.array([0, 5, 5, 1, 1, 1, 0, 1])

print(a == 5)
print(np.sum(a == 5))
print(a == 1)
print(np.sum(a == 1))
print(a >= 1)
print(np.sum(a >= 1))

b = np.array([0, 1, 0, 0, 1, 1, 0, 1])
print(b[a == 5])

[False  True  True False False False False False]
2
[False False False  True  True  True False  True]
4
[False  True  True  True  True  True False  True]
6
[1 0]


In [85]:
def prior_prob(ytrain, label):
    total_examples = ytrain.shape[0]
    class_example = np.sum(ytrain == label)
    return (class_example)/float(total_examples)

In [86]:
def condition_prob(xtrain, ytrain, feature_col_index, feature_col_val,label): # label means class c
    xtrain_filtered_by_label = xtrain[ytrain == label]
    numerator = np.sum(xtrain_filtered_by_label[:,feature_col_index] == feature_col_val)
    denominator = np.sum(ytrain == label)
    return numerator/float(denominator)

5. Compute Posterior probability for each test example and make predictions

In [95]:
def predict(xtrain, ytrain, xtest):
    """here xtest is a single testing point with n features """

    posterior_prob_list = [] # this will hold list of prob for all classes and a given testing point

    # compute posterior prob for each class
    for label in np.unique(ytrain):
        # Posterior_c = likelihood * Prior
        # Note: it's also divided by marginal prob(holds "same" value across all labels) which we can ignore because we need to compute argmax of Posterior_c across all labels)
        feature_idx = 0
        prior_prob_value =  prior_prob(ytrain, label)
        likelihood = 1.0
        for feature_value in xtest:
            likelihood = likelihood * condition_prob(xtrain, ytrain, feature_idx, feature_value,label)
            feature_idx = feature_idx + 1
        
        post = likelihood * prior_prob_value
        posterior_prob_list.append(post)

    pred = np.argmax(posterior_prob_list)
    return pred
            

In [101]:
output = predict(xtrain, ytrain, xtest[0])
print(output)
print(ytest[0])
output = predict(xtrain, ytrain, xtest[1])
print(output)
print(ytest[1])
output = predict(xtrain, ytrain, xtest[2])
print(output)
print(ytest[2])

0
0
0
0
1
1


In [102]:
def score(xtrain,ytrain, xtest, ytest):
    pred = []

    for i in range(xtest.shape[0]):
        pred_label = predict(xtrain,ytrain,xtest[i])
        pred.append(pred_label)
    
    pred = np.array(pred)
    accuracy = np.sum(pred == ytest)/ytest.shape[0]
    return accuracy 

In [104]:
print(score(xtrain, ytrain, xtest, ytest))

0.9993846153846154
